<a href="https://colab.research.google.com/github/sadiashoaib415/Quantum_Computer/blob/main/QKD_BB84_Protocol_with_QEBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.6 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import random

backend = Aer.get_backend("aer_simulator")

# generate random bits + bases
n = 20
alice_bits  = [random.randint(0,1) for _ in range(n)]
alice_basis = [random.choice(["Z","X"]) for _ in range(n)]
bob_basis   = [random.choice(["Z","X"]) for _ in range(n)]
bob_bits    = []

for bit, a_basis, b_basis in zip(alice_bits, alice_basis, bob_basis):
    qc = QuantumCircuit(1,1)

    # Alice prepares
    if a_basis == "X":
        qc.h(0)
    if bit == 1:
        qc.x(0)

    # Bob measures
    if b_basis == "X":
        qc.h(0)
    qc.measure(0,0)

    qc_t = transpile(qc, backend)
    result = backend.run(qc_t, shots=1).result()
    bob_bits.append(int(list(result.get_counts().keys())[0]))

# sift key (bases match)
sift = [i for i in range(n) if alice_basis[i]==bob_basis[i]]
key_A = [alice_bits[i] for i in sift]
key_B = [bob_bits[i] for i in sift]

# QBER
errors = sum(1 for i in range(len(key_A)) if key_A[i]!=key_B[i])
qber = errors / max(len(key_A),1)

print("Alice key:", key_A)
print("Bob key:  ", key_B)
print("QBER:", qber)


Alice key: [0, 1, 1, 0, 0, 1, 0, 1, 1, 1]
Bob key:   [0, 1, 0, 0, 0, 0, 0, 0, 1, 1]
QBER: 0.3


“Why is QBER not zero? There is no Eve!”

This is the important insight

This code prepares qubits in the wrong order

In BB84, the correct preparation is:

Encode bit value

Then choose basis

Corrected BB84 Code (Google Colab / Qiskit 1.x)

In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import random

backend = Aer.get_backend("aer_simulator")

# number of qubits
n = 20

# Alice and Bob choices
alice_bits  = [random.randint(0,1) for _ in range(n)]
alice_basis = [random.choice(["Z","X"]) for _ in range(n)]
bob_basis   = [random.choice(["Z","X"]) for _ in range(n)]
bob_bits    = []

for bit, a_basis, b_basis in zip(alice_bits, alice_basis, bob_basis):
    qc = QuantumCircuit(1,1)

    # ✅ Alice prepares (CORRECT ORDER)
    if bit == 1:
        qc.x(0)
    if a_basis == "X":
        qc.h(0)

    # Bob measures
    if b_basis == "X":
        qc.h(0)
    qc.measure(0,0)

    qc_t = transpile(qc, backend)
    result = backend.run(qc_t, shots=1).result()
    bob_bits.append(int(list(result.get_counts().keys())[0]))

# sift key (bases match)
sift = [i for i in range(n) if alice_basis[i] == bob_basis[i]]
key_A = [alice_bits[i] for i in sift]
key_B = [bob_bits[i] for i in sift]

# QBER calculation
errors = sum(1 for i in range(len(key_A)) if key_A[i] != key_B[i])
qber = errors / max(len(key_A), 1)

print("Alice key:", key_A)
print("Bob key:  ", key_B)
print("QBER:", qber)


Alice key: [1, 1, 0, 1, 0, 1, 0, 1, 0, 0]
Bob key:   [1, 1, 0, 1, 0, 1, 0, 1, 0, 0]
QBER: 0.0


In ideal BB84 (no Eve, no noise):

Matching bases → perfect correlation

QBER should be ~0

Any non-zero QBER indicates:

Eavesdropping or

Noise or

Bug